In [1]:
import requests
import pandas as pd
from tqdm import tqdm
import os

In [3]:
# Get token

FOSSOLOGY_SERVER = "http://localhost/repo/api/v1"
FOSSOLOGY_USER = "fossy"
FOSSOLOGY_PASSWORD = "fossy"
TOKEN_NAME = "fossy_token_7"
TOKEN_EXPIRE = "2023-07-30"
token = requests.post(
    url=FOSSOLOGY_SERVER + "/tokens",
    json={
        "username": FOSSOLOGY_USER, 
        "password": FOSSOLOGY_PASSWORD,
        "token_expire": TOKEN_EXPIRE,
        "token_name": TOKEN_NAME,
        "token_scope": "write",
        },
    ).json()['Authorization']

In [4]:
uploads = requests.get(
    url=FOSSOLOGY_SERVER + "/uploads",
    headers={
        "Accept": "application/json",
        "Authorization": token
        },
    ).json()

upload_ids = [upload['id'] for upload in uploads]
upload_ids

[8, 9, 10, 11, 12, 13, 14]

In [ ]:
source_dir = 'repositories'
for source_file in os.listdir(source_dir):
    # Upload file to FOSSology
    file_path = os.path.join(source_dir, source_file)
    upload_file = requests.post(
        url=FOSSOLOGY_SERVER + "/uploads",
        headers={
        'Accept': 'application/json',
        'Content-Type': 'multipart/form-data',
        'groupName': 'fossy',
        'ignoreScm': 'false',
        'applyGlobal': 'false',
        'public': 'protected',
        'uploadDescription': 'Upload from fossologyAPI',
        'folderId': '1',
        'Authorization': token
        },
        json={
            'fileInput': file_path,
            
        }
    ).json()

upload_file

In [5]:
copyrights = requests.get(
    url=FOSSOLOGY_SERVER + "/uploads/" + str(upload_ids[3]) + "/copyrights",
    headers={
        "Accept": "application/json",
        "Authorization": token
        },
)

In [ ]:
copyrights

In [7]:
copyrights = copyrights.json()

In [8]:
copyrights[0]['filePath']

['kubernetes-master.zip/kubernetes-master/LICENSE',
 'kubernetes-master.zip/kubernetes-master/staging/src/k8s.io/apiextensions-apiserver/LICENSE',
 'kubernetes-master.zip/kubernetes-master/staging/src/k8s.io/apimachinery/LICENSE',
 'kubernetes-master.zip/kubernetes-master/staging/src/k8s.io/kube-scheduler/LICENSE',
 'kubernetes-master.zip/kubernetes-master/staging/src/k8s.io/cluster-bootstrap/LICENSE',
 'kubernetes-master.zip/kubernetes-master/staging/src/k8s.io/kubelet/LICENSE',
 'kubernetes-master.zip/kubernetes-master/staging/src/k8s.io/component-base/LICENSE',
 'kubernetes-master.zip/kubernetes-master/staging/src/k8s.io/apiserver/LICENSE',
 'kubernetes-master.zip/kubernetes-master/staging/src/k8s.io/api/LICENSE',
 'kubernetes-master.zip/kubernetes-master/staging/src/k8s.io/kube-controller-manager/LICENSE',
 'kubernetes-master.zip/kubernetes-master/staging/src/k8s.io/dynamic-resource-allocation/LICENSE',
 'kubernetes-master.zip/kubernetes-master/staging/src/k8s.io/code-generator/LIC

In [9]:
copyrights_df = pd.DataFrame(columns=['file_path', 'copyright', 'falsePositive'])

In [10]:
for i in tqdm(range(len(copyrights))):
    copyright = copyrights[i]['copyright']
    paths = copyrights[i]['filePath']
    falsePositive = 0
    for j in range(len(paths)):
        current_path = paths[j].split('.zip/')[1]
        row = pd.Series({
                    'file_path': current_path,
                    'copyright': copyright,
                    'falsePositive': falsePositive
                })
        copyrights_df.loc[len(copyrights_df)] = row

100%|██████████| 577/577 [00:34<00:00, 16.82it/s] 


In [11]:
len(copyrights_df)

16497

In [12]:
copyrights_df.head(10)

,file_path,copyright,falsePositive
0,kubernetes-master/LICENSE,copyright owner or by an individual or Legal E...,0
1,kubernetes-master/staging/src/k8s.io/apiextens...,copyright owner or by an individual or Legal E...,0
2,kubernetes-master/staging/src/k8s.io/apimachin...,copyright owner or by an individual or Legal E...,0
3,kubernetes-master/staging/src/k8s.io/kube-sche...,copyright owner or by an individual or Legal E...,0
4,kubernetes-master/staging/src/k8s.io/cluster-b...,copyright owner or by an individual or Legal E...,0
5,kubernetes-master/staging/src/k8s.io/kubelet/L...,copyright owner or by an individual or Legal E...,0
6,kubernetes-master/staging/src/k8s.io/component...,copyright owner or by an individual or Legal E...,0
7,kubernetes-master/staging/src/k8s.io/apiserver...,copyright owner or by an individual or Legal E...,0
8,kubernetes-master/staging/src/k8s.io/api/LICENSE,copyright owner or by an individual or Legal E...,0
9,kubernetes-master/staging/src/k8s.io/kube-cont...,copyright owner or by an individual or Legal E...,0


In [13]:
copyrights_df.to_csv(os.path.join('fossologyAPI_CSVs', 'kubernetes-master.csv'))